<a href="https://colab.research.google.com/github/wangbxj1234/xformerforpt/blob/main/local_linear_Attention%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
x = torch.rand(3,2)
print(x)
x[0][0]

tensor([[0.2503, 0.3117],
        [0.6553, 0.7998],
        [0.9133, 0.7686]])


tensor(0.2503)

In [12]:
K=x.sum(dim=1)
K

tensor([0.5620, 1.4550, 1.6819])

In [11]:
y=torch.ones(3)
y[0]=2
print(y)

tensor([2., 1., 1.])


In [13]:
torch.einsum("l,l->l", y,K)

tensor([1.1239, 1.4550, 1.6819])

In [10]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn

x = torch.rand(64,6)####n*c
xyz = torch.rand(64,3) #n*3
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [N, C]
        dst: target points, [M, C]
    Output:
        dist: per-point square distance, [ N, M]
    """
    return torch.sum((src[:, None] - dst[None,:]) ** 2, dim=-1)

def index_points(points, idx):#从n个坐标中按照index提取s个坐标或者s*k个坐标，进行grouping
    """
    Input:
        points: input points data, [N, C]
        idx: sample index data, [N, [K]]
    Return:
        new_points:, indexed points data, [N, [K], C] 
    """
    raw_size = idx.size()#这里是torch size 相当于 numpy的shape
    idx = idx.reshape(-1) # N*K.
    res = torch.gather(points, 0, idx[..., None].expand(-1, points.size(-1))) #先把idx的第2维复制到=C,再用gather按索引提取对应点出来。
    return res.reshape(*raw_size, -1)#就是idx的shape（2维或者三维），再加上最后一维c，理论上这个-1也可以写成'points.size(-1)'吧。

dists = square_distance(xyz, xyz)
knn_idx = dists.argsort()[:, :16]  # n x k
print(knn_idx.shape)
index_points(x, knn_idx).shape   #n*k*c

torch.Size([64, 16])


torch.Size([64, 16, 6])

In [1]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [N, C]
        dst: target points, [M, C]
    Output:
        dist: per-point square distance, [ N, M]
    """
    return torch.sum((src[:, None] - dst[None,:]) ** 2, dim=-1)

def index_points(points, idx):#从n个坐标中按照index提取s个坐标或者s*k个坐标，进行grouping
    """
    Input:
        points: input points data, [N, C]
        idx: sample index data, [N, [K]]
    Return:
        new_points:, indexed points data, [N, [K], C] 
    """
    raw_size = idx.size()#这里是torch size 相当于 numpy的shape
    idx = idx.reshape(-1) # N*K.
    res = torch.gather(points, 0, idx[..., None].expand(-1, points.size(-1))) #先把idx的第2维复制到=C,再用gather按索引提取对应点出来。
    return res.reshape(*raw_size, -1)#就是idx的shape（2维或者三维），再加上最后一维c，理论上这个-1也可以写成'points.size(-1)'吧。




class local_linear_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes,k):
        super(local_linear_Attention, self).__init__()
        self.n_heads =8
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        ####加入位置编码
        #self.p_proj_q = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        #self.p_proj_k = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        ####
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6
        self.k = k

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           p_s=p[PN:int(offset),:]
           p_sub.append(p_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           S=tgt_len
           ###
           dists = square_distance(p_s, p_s)
           knn_idx = dists.argsort()[:, :self.k]  # n x k
           ###
           Q = self.query_projection(x_s).view(L, self.n_heads, -1)
           K = index_points(self.key_projection(x_s), knn_idx).view(S, self.k,self.n_heads, -1)
           ####加入位置编码
           #pe_q = self.p_proj_q(p_s).view(L, self.n_heads, -1)
           #pe_k = self.p_proj_k(p_s).view(S, self.n_heads, -1)
           #queries = queries+pe_q
           #keys = keys+pe_k
           ####
           V = index_points(self.value_projection(x_s), knn_idx).view(S, self.k,self.n_heads, -1)

           # 2. Non-negative projection
           Q = self.kernel_method(Q)
           K = self.kernel_method(K)

           KV = torch.einsum("skhd,skhm->hmd", K, V)

           # Compute the normalizer
           Z = 1/(torch.einsum("lhd,lhd->lh", Q, K.sum(dim=1))+self.eps)

           # Finally compute and return the new values
           V = torch.einsum("lhd,hmd,lh->lhm", Q, KV, Z)


           output = V.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out


In [2]:
o=[ 838, 2088, 2317, 3567, 4541, 5561, 6427, 7077]
o=torch.IntTensor(o)
x = torch.rand(7077, 128)
p = torch.rand(7077, 3)

In [3]:
planes=128
transformer2 = local_linear_Attention(planes,k=16)
right_res = transformer2([p, x, o])
print(right_res.shape)

torch.Size([7077, 128])
